tgb - 4/10/2020 - Train 4 networks on new aquaplanet preprocessed data:  
1. Standard (RH,T-TNS,LHF) input scaling
2. Same with radiation as additional output
3. Repeat with continuous vertical interpolation layer

In [1]:
from cbrain.imports import *
from cbrain.cam_constants import *
from cbrain.utils import *
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow_probability as tfp
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
# import cartopy.crs as ccrs
import matplotlib.ticker as mticker
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
# from climate_invariant import *
from tensorflow.keras import layers
import datetime
from climate_invariant_utils import *
import yaml

/data11/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


ModuleNotFoundError: No module named 'tensorflow_probability'